# Acquire ACS Data Over Serial WITHOUT Knowledge From the Device File
For this example, you will need to connect an ACS to your computer via serial. 
The sensor should be on before running this notebook. Note that the ACS is in picoDOS for the first 10 seconds after it is powered on and may not register on a serial port.

Note: In this example, absorption and attenuation will look bad because the ACS is running in air.


In [4]:
from datetime import datetime, timezone

from acspype import ACSStream
from acspype.processing import convert_sn_str
import acspype.qaqc as acsqaqc
from acspype.utils import find_acs_port # This is a convenience function that finds the first port that has an ACS attached to it.

In [5]:
num_loops = 5  # Acquire X packets, and then stop. Otherwise, the while loop will run forever.

In [6]:
port = find_acs_port()  # Find the port that has an ACS attached to it.
port = 'COM5'
with ACSStream(port) as acss: # Open a serial object.
    i = 0
    while True:  # Continuously read the stream until a full packet is found.
        acss.read_stream()
        acs_pkt = acss.find_packet()
        if acs_pkt.full_packet is not None:
            parsed_packet = acss.parse_packet(acs_packet = acs_pkt) # Parse the packet.
            
            # Run the gap and syntax tests.
            flag_gap = acsqaqc.gap_test(now = datetime.now(timezone.utc), 
                                        time_stmp = acs_pkt.daq_time, 
                                        buffer_length= len(acss._buffer), # The bytes in the serial buffer can be accessed with the _buffer attribute. This is really the only instance where you would need access to the buffer contents outside of the ACSStream class.
                                        record_length = parsed_packet.record_length)
            flag_syntax = acsqaqc.syntax_test(full_packet = acs_pkt.full_packet)
            
            # Print some of the results.
            print(f"---------- Packet {i} ----------")
            print(f"Acquisition Time: {parsed_packet.daq_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'}")
            print(f"Acquired Packet From: {convert_sn_str(parsed_packet.sn_int)}")
            print(f"Elapsed Time (ms): {parsed_packet.elapsed_time}")
            print(f"Absorption Signal Counts: {parsed_packet.a_signal}")
            if flag_gap == 1:
                print('Gap Test: Passed')
            elif flag_gap == 4:
                print('Gap Test: Failed')
            if flag_syntax == 1:
                print('Syntax Test: Passed')
            elif flag_syntax == 4:
                print('Syntax Test: Failed')
            print('\n')
            
            i += 1
            if i == num_loops:
                break

---------- Packet 0 ----------
Acquisition Time: 2025-05-04T18:01:15.143Z
Acquired Packet From: ACS-00011
Elapsed Time (ms): 623434
Absorption Signal Counts: (507, 567, 641, 734, 847, 971, 1107, 1277, 1466, 1713, 1993, 2268, 2565, 2888, 3248, 3604, 4002, 4403, 4846, 5360, 5901, 6438, 7059, 7663, 8281, 8914, 9513, 10217, 11011, 11754, 12512, 13373, 14258, 15323, 16398, 17413, 18470, 19433, 20521, 21553, 22390, 23345, 24241, 25210, 26386, 27192, 29275, 30147, 30985, 31689, 32388, 32991, 33651, 34296, 34610, 34773, 34892, 34940, 34886, 34751, 34336, 33651, 32879, 31985, 31548, 31200, 30260, 29247, 28248, 27103, 26005, 24914, 23676, 22274, 20923, 19787, 18725, 17554, 16370, 15253, 14142, 13130, 12197, 11233)
Gap Test: Passed
Syntax Test: Passed


---------- Packet 1 ----------
Acquisition Time: 2025-05-04T18:01:15.400Z
Acquired Packet From: ACS-00011
Elapsed Time (ms): 623685
Absorption Signal Counts: (503, 561, 635, 729, 842, 964, 1103, 1273, 1465, 1713, 1995, 2269, 2568, 2889, 3250, 3605